## Graphs of tf.functions

通过 `Model.fit()` 我们可以生成计算图。当然我们也可以使用 `tf.function` 标准来将 Python 的计算转换成高效的 TensorFlow 图。

在 TensorBoard 可视化中，我们可以使用 TensorFlow Summary Trace API 来记录自动图函数。

使用 API 的正确方式：

- 定义函数并用  `tf.function` 来进行注解。
- 在调用时执行 `tf.summary.trace_on()`
- 传入 `profiler=True` 可以剖析内存和 CPU 的执行信息。
- 调用 `tf.summary.trace_export()` 存储日志数据。

In [1]:
from datetime import datetime

import tensorflow as tf
from tensorflow import keras

In [3]:
%load_ext tensorboard

The tensorboard module is not an IPython extension.


In [2]:
# The function to be traced
@tf.function
def my_func(x, y):
    # A simple hand-rolled layer
    return tf.nn.relu(tf.matmul(x, y))

# Set up logging
stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

# Sample data for your function
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export()
tf.summary.trace_on(graph=True, profiler=True)

# call only one tf.function when tracing
z = my_func(x, y)
with writer.as_default():
    tf.summary.trace_export(
        name='my_func_trace',
        step=0,
        profiler_outdir=logdir
    )

在命令行中执行

```bash
tensorboard --logdir logs/func
```
就可以在浏览器查看计算信息了。包括内存使用时间和 CPU 执行时间等。

<img src="./tensorboard-func.png">